In [5]:
!pip install opencv-python

In [6]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os

In [7]:
URL = 'https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded'
response = requests.get(URL)

In [8]:
store_ds_path = ''
fname = os.path.join(store_ds_path, 'data.zip')
with open(fname, 'wb') as f:
    f.write(response.content)

In [9]:
with zipfile.ZipFile(fname, 'r') as f:
    f.extractall()

In [10]:
def loader(path):
    im = cv2.imread(path)
    return im

In [11]:

import torchvision
from torchvision import models, datasets, transforms

path = os.path.join(store_ds_path,'Plant_leave_diseases_dataset_without_augmentation')
def resize_image(x):
    return cv2.resize(x, (150, 150))

# Define the transformation pipeline without using lambda functions
tfms = transforms.Compose([
    resize_image,  # Use the custom function to resize images
    transforms.ToTensor(),
    transforms.ColorJitter(brightness=0.4, contrast=0, saturation=0, hue=0),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomAffine(
        degrees=(0, 360),
        translate=(0.1, 0.1),
        scale=(0.5, 1)
    )
])


In [12]:
ds = datasets.ImageFolder(path, loader=loader, transform=tfms)

In [13]:
def beautify_labels(label):
    bucket = re.findall(r'[^,_() ]+', label)[:3]
    bucket = [x.capitalize() for x in bucket]
    return ''.join(bucket)

In [14]:
label_names = list(map(beautify_labels, ds.classes))

In [15]:
label2idx = {key: val for val, key in enumerate(label_names)}
idx2label = {val: key for key, val in label2idx.items()}

In [16]:
import torch
train_ds, val_ds, test_ds  = torch.utils.data.random_split(ds, (45448, 5000, 5000))

In [18]:
!pip install pytorch-lightning

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, datasets, transforms
import pytorch_lightning as pl
import torchmetrics as tm
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, Callback
from pytorch_lightning.loggers import TensorBoardLogger

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [20]:

from torch.utils.data import DataLoader



class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(num_classes=39)
        self.accuracy = tm.Accuracy(task='multiclass', num_classes=39)
        self.loss_func = nn.CrossEntropyLoss()

    def forward(self, x):
        out = self.model(x)
        return out

    def training_step(self, batch, batch_idx):
        data, label = batch
        out = self(data)
        acc = self.accuracy(out, label)
        loss = self.loss_func(out, label)
        self.log('Loss/train', loss)
        self.log('Accuracy/train', acc)
        return loss


    def validation_step(self, batch, batch_idx):
        data, label = batch
        out = self(data)
        acc = self.accuracy(out, label)
        loss = self.loss_func(out, label)
        self.log('Loss/val', loss)  # Log validation loss
        return {'val_loss': loss, 'val_acc': acc}

    def on_validation_epoch_end(self):
        if 'val_loss' in self.trainer.callback_metrics:
            avg_loss = torch.stack([x['val_loss'] for x in self.trainer.callback_metrics['val_loss']]).mean()
            print(f'Overall Validation Loss: {avg_loss.item()}')
        if 'val_acc' in self.trainer.callback_metrics:
            avg_acc = torch.stack([x['val_acc'] for x in self.trainer.callback_metrics['val_acc']]).mean()
            print(f'Overall Validation Accuracy: {avg_acc.item()*100:.2f}%')

    def predict_step(self, batch, batch_idx):
        data, label = batch
        out = self(data)
        _, idx = torch.max(out, dim=1)
        return idx

    def test_step(self, batch, batch_idx):
        data, label = batch
        out = self(data)
        acc = self.accuracy(out, label)
        loss = self.loss_func(out, label)
        self.log('Loss/test', loss)
        self.log('Accuracy/test', acc)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=3e-3, weight_decay=0) # wd = 3e-3
        steps_per_epoch = math.ceil(len(train_ds)/64)
        scheduler = {"scheduler": optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, epochs=10, steps_per_epoch=steps_per_epoch),"interval": "step","frequency": 1,}
        return [optimizer], [scheduler]

    def train_dataloader(self):
        train_dl = DataLoader(train_ds, shuffle=True, batch_size=64, num_workers=2)
        return train_dl

    def val_dataloader(self):
        val_dl = DataLoader(val_ds, batch_size=64, num_workers=2, persistent_workers=True)
        return val_dl

    def test_dataloader(self):
        test_dl = DataLoader(test_ds, batch_size=64, num_workers=2)
        return test_dl

model = Model()

logger = pl.loggers.TensorBoardLogger('lightning_logs', name='baseline-resnet18')
trainer = pl.Trainer(accelerator='gpu', logger=logger, max_epochs=10, profiler='simple', callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="Loss/val"), LearningRateMonitor("step")])

trainer.fit(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ResNet             | 11.2 M
1 | accuracy  | MulticlassAccuracy | 0     
2 | loss_func | CrossEntropyLoss   | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.786    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.profilers.profiler:FIT Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                       

In [21]:
torch.save(model, 'model.pt')